In [15]:
library(Seurat)
seurat = readRDS('../initialSeurat_rhesus.RDS')

In [16]:
resultsA = read.table('211210LovA.csv', header = 1, stringsAsFactors = FALSE, sep = ',', row.names = 1)
resultsB = read.table('220110Lov.csv', header = 1, stringsAsFactors = FALSE, sep = ',', row.names = 1)
results = rbind(resultsA, resultsB)

In [17]:
seurat = subset(seurat, cells = colnames(seurat)[seurat$id %in% rownames(results)])

In [18]:
seurat

An object of class Seurat 
12565 features across 13366 samples within 1 assay 
Active assay: RNA (12565 features, 2000 variable features)
 2 dimensional reductions calculated: pca, umap

In [19]:
hash_mat = results[seurat$id,]
bcs_to_names = colnames(seurat)
names(bcs_to_names) = seurat$id
rownames(hash_mat) = bcs_to_names[rownames(hash_mat)]

In [20]:
seurat[['HTO']] = CreateAssayObject(counts = t(hash_mat))
#seurat = subset(seurat, cells =colnames(seurat)[seurat$n_genes > 1000])

In [21]:
seurat = subset(seurat, cells = colnames(seurat)[seurat@meta.data$nCount_HTO > 5 ])

In [22]:
options(repr.plot.res = 300)
library(MASS)
library(cluster)
library(outliers)
library(dplyr)
seurat$hash = NA
Idents(seurat) = 'orig'
bcs_to_fix = c()
for (curr in unique(seurat$orig)) {
    message(curr)
   sub =subset(seurat, idents =  curr)
   #sub =subset(sub, nCount_HTO > 50)

    if (length(colnames(sub)) > 20) {
        
        if (substring(curr, 1,2 ) %in% c('AB'))  {
        sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(2,3,4,5)]))
            }
        if (curr =='AC1')  {
        sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(1,6,7)]))
        }
        if (curr =='AC2')  {
        sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(8,9,10)]))
        }
        
        if (curr =='AC3')  {
            sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(1,6,7,8,9,10)]))
        }
        
        if (curr =='AC4')  {
            sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(1,6,7,8,9,10)]))
        }
        if (curr =='AC5')  {
            sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(1,6,7,8,9,10)]))
        }
        sub = NormalizeData(sub, assay = 'HTO', normalization.method = 'CLR', margin = 2)
        
        

        DefaultAssay(sub) = 'HTO'
        data = GetAssayData(sub)

        # k medioids
        meds = pam(t(data), k = dim(data)[1])
        calls = t(data)
        cluster_data = meds$medoids %>% as.matrix()
        

        for (i in 1:dim(cluster_data)[2]) {
            
  

            curr_med = which.max(cluster_data[,i])
            negs = meds$clustering[meds$clustering != curr_med]
            neg_vals = data[i, names(negs)]
            pos_vals = data[i, meds$clustering == curr_med]
            pos_vals = pos_vals[order(pos_vals)] 
            done = FALSE
            thresh = min(pos_vals)
            # remove outliers and set threshold
            while(!done & length(pos_vals) > 5) {
                done = TRUE
                avg = mean(pos_vals)
                diff1 = avg - min(pos_vals)
                diff2 = max(pos_vals) - avg
                test = grubbs.test(pos_vals, opposite = diff1 < diff2)
                if (test$p.value < 0.05) {
                    pos_vals = pos_vals[-1]
                    done = FALSE
                }
                thresh = min(pos_vals)

            }
             if (curr == 'AB1' & i == 3) {
                thresh = .6
            }
            
             if (curr == 'AB2' & i == 3) {
                thresh = .3
            }
            
            if (curr == 'AB3') {
                thresh = 1
            }
            if (curr == 'AB3' & i == 2) {
                thresh = 1.5
            }
             if (curr == 'AB3' & i == 4) {
                thresh = 1.5
            }
            
            if (curr == 'AB4') {
                thresh = 2
            }
            
            if (curr == 'AC3' & i == 5) {
                thresh = 2
            }
            if (curr == 'AC2' & i == 2) {
                thresh = 1
            }
            if (curr == 'AC4' & i == 5) {
                thresh = 1
            }
             if (curr == 'AC4' & i == 5) {
                thresh = 1
            } 
            if (curr == 'AC5' & i == 5) {
                thresh = 1
            }
            print(thresh)
            
            
            
            calls[,i] = as.numeric(calls[,i] >= thresh)
            
 


        }
            assign = apply(calls, 1, which.max)
            assign = rownames(sub)[assign]
            rsums = rowSums(calls)
            print(curr_med)
            assign[rsums > 1] = 'Doublet'
            assign[rsums == 0] = 'Negative'
            
            if (substring(curr, 1, 2) == 'AB') {
                needToFix = rownames(calls)[(rowSums(calls)) == 2 & (calls[,2] == 1) & (calls[,4 ] == 1)]
                bcs_to_fix = c(bcs_to_fix,needToFix)
                assign[which(rownames(calls) %in% needToFix)] = 'NeedToFix'

            }
        
            pheatmap::pheatmap(t(data), annotation_row = data.frame(row.names = colnames(data), id = factor(assign)),show_rownames = FALSE, filename = paste0(curr, '.png'))
            seurat@meta.data[colnames(sub), 'hash'] = assign


    }
}

AB1

Normalizing across cells



[1] 2.183764
[1] 0.8549858
[1] 0.6
[1] 1.266306
AB1_142 
      4 


AB2

Normalizing across cells



[1] 3.14469
[1] 0.2747699
[1] 0.3
[1] 1.660865
AB2_794 
      3 


AB3

Normalizing across cells



...

[1] 2.284016
[1] 1.942184
[1] 1.549514
[1] 2.075925
[1] 2
[1] 2.075925
AC3_7 
    4 


AC4

Normalizing across cells



[1] 2.537923
[1] 2.266018
[1] 1.882588
[1] 1.866041
[1] 1
[1] 2.21777
AC4_2567 
       2 


AC5

Normalizing across cells



[1] 2.643779
[1] 2.421515
[1] 1.96454
[1] 2.294031
[1] 1
[1] 2.269497
AC5_2421 
       4 


In [24]:
fillin = apply(hash_mat[bcs_to_fix,], 1, which.max)
seurat@meta.data[bcs_to_fix, 'hash'] = colnames(hash_mat)[fillin]

In [25]:
table(seurat@meta.data$orig, seurat@meta.data$hash)

     
      Doublet HTO1.GTCAACTCTTTAGCG HTO10.ATTGACCCGCGTTAG HTO2.TGATGGCCTATTGGG
  AB1       8                    0                     0                   29
  AB2      19                    0                     0                   21
  AB3      24                    0                     0                  230
  AB4       2                    0                     0                   97
  AC1       0                   28                     0                    0
  AC2       1                    0                    48                    0
  AC3      74                  203                   430                    0
  AC4     106                  266                   507                    0
  AC5     142                  272                   619                    0
     
      HTO3.TTCCGCCTCTCTTTG HTO4.AGTAAGTTCAGCGTA HTO5.AAGTATCGTTTCGCA
  AB1                   25                  425                   67
  AB2                   17                  343                   70
 

In [56]:
saveRDS(seurat@meta.data, 'hashed_seurat_meta.RDS')
#saveRDS(seurat, 'hashed_seurat.RDS')
